In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
protection_keys = [
    "без средств защиты",
    "кожух без звукопоглоитителя",
    "кожух со звукопоглотителем",
    "экран металлический сплошной",
    "экран с отверстиями для вентиляции",
    "экран с верхним большим отверстием",
    "оргалитовый экран сплошной",
    "экран металлический сплошной + кожух со звукопоглоитителем",
]

In [ ]:
f = np.array([31.5, 63, 125, 250, 500, 1000, 2000, 4000, 8000], dtype=np.float32)

background_noise = np.array([61.9, 56.3, 44.9, 48.3, 33.4, 27.6, 25.5, 23.9, 25.6, 40.4], dtype=np.float32)
limit = np.array([86.0, 71.0, 61.0, 54.0, 49.0, 45.0, 42.0, 40.0, 38.0, 50.0], dtype=np.float32)

noises = {
    protection_keys[0]: np.array([62.1, 57.4, 58.7, 75.0, 86.6, 91.1, 96.1, 70.1, 44.1, 98.2], dtype=np.float32),
    protection_keys[1]: np.array([60.1, 55.9, 57.4, 67.9, 76.6, 90.3, 95.7, 62.3, 33.5, 97.5], dtype=np.float32),
    protection_keys[2]: np.array([61.9, 57.7, 56.7, 65.0, 72.5, 85.2, 90.5, 53.5, 27.5, 92.4], dtype=np.float32),
    protection_keys[3]: np.array([60.1, 56.3, 56.4, 71.6, 82.7, 86.2, 90.4, 61.9, 31.0, 92.7], dtype=np.float32),
    protection_keys[4]: np.array([71.8, 69.5, 61.5, 72.9, 84.8, 86.3, 90.7, 69.5, 34.7, 93.0], dtype=np.float32),
    protection_keys[5]: np.array([60.5, 59.8, 64.2, 73.7, 86.0, 90.7, 97.9, 68.7, 39.7, 99.6], dtype=np.float32),
    protection_keys[6]: np.array([67.9, 64.2, 56.9, 72.1, 83.2, 86.2, 90.4, 62.5, 30.8, 92.7], dtype=np.float32),
    protection_keys[7]: np.array([59.6, 56.8, 55.0, 59.4, 70.2, 72.5, 76.1, 44.2, 25.7, 80.5], dtype=np.float32)
}

In [ ]:
protection_effectiveness = { 
    protection_key: (noises["без средств защиты"] - noises[protection_key]) 
    for protection_key in protection_keys
}

noises_without_bg = { 
    protection_key: (noises[protection_key] - background_noise) 
    for protection_key in protection_keys
}

noises_limit_exceeding = {
    protection_key: (noises[protection_key] - limit) 
    for protection_key in protection_keys
}

noises_without_bg_limit_exceeding = { 
    protection_key: (noises_without_bg[protection_key] - limit) 
    for protection_key in protection_keys
}

In [ ]:
def get_sound_pressure(row: np.ndarray):
    return row[:-1]

def get_sound_level(row: np.ndarray):
    return row[-1]

# 1. Расчеты в табличном виде (для копирования в Word)

In [ ]:
def build_table(data):
    df = pd.DataFrame(
        data={name: data[name] for name in data.keys()},
        index=[str(v) for v in f] + ["Уровни звука"]
    ).T

    df.index.name = "Средства защиты"

    return df

In [ ]:
data = noises_without_bg_limit_exceeding

table = build_table(data)
pd.set_option('display.precision', 3)

# убираем уровни звука
table = table.iloc[:, :-1]
# 

table

# 2. Расчеты на одном графике

In [ ]:
def build_sound_pressure_common_plot(data, ax, octave_scale=False, ylim=None, ystep=None):
    pressures = {
        key: get_sound_pressure(value)
        for key, value in data.items()
    }

    markers =       ['x', 's', '^', None, None, 'o', None, '^']
    linestyles =    ['-', '-', '-', '-', '--', '-', '-', '-']
    colors =        [0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5]
    colors = [[color] * 3 for color in colors]

    keys = list(data.keys())
    order = [0, 1, 2, 4, 5, 6, 3, 7]

    for i in order:
        ax.plot(
            np.arange(len(f)) if octave_scale else f, 
            pressures[keys[i]], 
            color=colors[i], 
            marker=markers[i],
            linestyle=linestyles[i],
            ms=10,
            label=keys[i]
        )
    
    if octave_scale:
        ax.set_xticks(np.arange(len(f)))
        ax.set_xticklabels(f.astype(int)) 

    ax.set_xlabel("Частота, Гц")
    ax.set_ylabel("Уровень звукового давления, дБ")
    ax.grid(True, linestyle='dashed')

    if ylim:
        ax.set_ylim(ylim)
        if ystep != None:
            ax.set_yticks(np.arange(ylim[0], ylim[1] + 1e-6, ystep))
    
    ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 10))
data = noises_without_bg_limit_exceeding

# zero line
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

build_sound_pressure_common_plot(
    data, 
    ax,
    octave_scale=True,
    ylim=(-100, 40),
    ystep = 10,
    )

plt.show()

# 3. Расчеты на отдельных графиках

In [ ]:
def get_subplot_idxs(rows, cols, idx):
    return (idx // cols, idx % cols)

def build_sound_pressure_plots(data, axes, rows, cols, label, limit_curve=False):
    pressures = {
        key: get_sound_pressure(value)
        for key, value in data.items()
    }

    limit_pressures = get_sound_pressure(limit)

    for idx, protection_key in enumerate(protection_keys):
        i, j = get_subplot_idxs(rows, cols, idx)

        axes[i, j].plot(f, pressures[protection_key], color='blue', marker='o', ms=5, label=label)
        axes[i, j].set_title(protection_key)
        axes[i, j].set_xlabel("Частота, Гц")
        axes[i, j].set_ylabel("Уровень звукового давления, дБ")
        axes[i, j].grid(True, linestyle='dashed')

        if limit_curve:
            axes[i, j].plot(f, limit_pressures, color='red', marker='o', ms=5, label='предельный уровень')
        
        axes[i, j].legend()

In [ ]:
data = protection_effectiveness
title = "Эффективность средств защиты"
label = "уровень звукового давления"

fig, axes = plt.subplots(4, 2, figsize=(15, 20))

# for protection_efficiency
""" for idx, protection_key in enumerate(protection_keys):
    i, j = get_subplot_idxs(4, 2, idx)
    axes[i, j].plot(f, [0 for i in range(len(f))], color='red', ms=5, label='предельный уровень') """

build_sound_pressure_plots(data, axes, 4, 2, label=label, limit_curve=False)

plt.suptitle(f"{title}\n", fontsize=20)
plt.tight_layout()
plt.show()